You have to work on the [University dataset](https://drive.google.com/drive/folders/1Hs3nRtK_F3h8eg59B4-TD1DEua6g8Klv?usp=sharing). It contains three different university rankings:
*  The Times Higher Education World University Ranking, shortly *Times*,
*  the Academic Ranking of World Universities, shortly *Shanghai*,
*  the Center for World University Rankings, shortly *cwur*.

**Notes**

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com).

**Imports**

Imports are limited to standard library modules and *pandas*.

In [440]:
import os
import pandas as pd
from collections import Counter
from itertools import combinations

The project follows a functional approach for the implementation.

**Load Ranking Data**

The "-" character is used as a NA value in some of the files.

I also convert all the ranks to strings to avoid reconverting them later while parsing and remove the '=' character which is meaningless for the required computations.

In [383]:
ranks = {}
for ranking in ['times', 'cwur', 'shanghai']:
    ranks[ranking] = pd.read_csv(os.path.join('data', f'{ranking}Data.csv'), 
                                 converters={'world_rank': lambda x: str(x).replace('=', '')},
                                 na_values='-')

## For each university, extract from the *times* dataset the most recent and the least recent data, obtaining two separate dataframes

I tested (with `%%timeit` magic command) and it was 5-6 times faster to obtain the indexes of the oldest and newest data points and then subset rather than to order the dataframe by year and take, respectively, the first and last data points in the group.

I can use `idxmin` and `idxmax` since it's guaranteed that there's only one data point per year per university.

In [451]:
grouped_times = ranks['times'].groupby('university_name')
oldtimes = ranks['times'].loc[grouped_times.year.idxmin()].reset_index(drop=True)
newtimes = ranks['times'].loc[grouped_times.year.idxmax()].reset_index(drop=True)

In [452]:
oldtimes

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,NaN,"35,569",17.0,1%,NaN,2016
1,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,NaN,"17,422",15.9,15%,48 : 52,2012
2,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,NaN,"16,099",24.2,17%,32 : 68,2012
3,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011
4,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,NaN,"9,252",19.2,18%,48 : 52,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,42,École Normale Supérieure,France,66.8,44.9,48.2,95.7,30.7,68.6,"2,400",7.9,20%,46 : 54,2011
814,100,École Normale Supérieure de Lyon,France,51.1,37.6,34.4,88.8,26.1,57.0,"2,218",8.0,14%,49 : 51,2011
815,39,École Polytechnique,France,57.9,77.9,56.1,91.4,NaN,69.5,"2,429",4.8,30%,18 : 82,2011
816,48,École Polytechnique Fédérale de Lausanne,Switzerland,55.0,100.0,56.1,83.8,38.0,66.5,"9,666",10.5,54%,27 : 73,2011


In [453]:
newtimes

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,NaN,"35,569",17.0,1%,NaN,2016
1,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,NaN,"17,422",15.9,15%,48 : 52,2016
2,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,NaN,"16,099",24.2,17%,32 : 68,2016
3,106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
4,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,NaN,"9,252",19.2,18%,48 : 52,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,"2,400",7.9,20%,46 : 54,2016
814,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,NaN,"2,218",8.0,14%,49 : 51,2016
815,101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,"2,429",4.8,30%,18 : 82,2016
816,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,"9,666",10.5,54%,27 : 73,2016


## For each university, compute the improvement in `income` between the least recent and the most recent data points

In [105]:
merged = oldtimes.merge(newtimes, on='university_name', suffixes=('_old', '_new'))

In [108]:
merged['income_diff'] = merged.apply(lambda row: float(row.income_new) - float(row.income_old), axis=1) 

In [109]:
income_diffs = merged[['university_name', 'income_diff']]
income_diffs

,university_name,income_diff
0,AGH University of Science and Technology,NaN
1,Aalborg University,7.3
2,Aalto University,-0.3
3,Aarhus University,6.8
4,Aberystwyth University,-4.2
...,...,...
813,École Normale Supérieure,6.4
814,École Normale Supérieure de Lyon,5.6
815,École Polytechnique,NaN
816,École Polytechnique Fédérale de Lausanne,27.4


## Find the university with the largest increase computed in the previous point

I use a different approach to find the maximum here to allow for possible ties.

In [454]:
max_income = income_diffs[income_diffs['income_diff'] == income_diffs['income_diff'].max()]

In [455]:
max_income

,university_name,income_diff
428,TU Dresden,67.8


## For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for *Aarhus University* the value is 122-73=49). Notice that some rankings are expressed as a range

In [365]:
def uniform_df(df):
    if 'institution' in df.columns:
        return df.rename(columns={'institution': 'university_name'})
    return df

def get_recent(df):
    df = uniform_df(df)
    grouped = df.groupby('university_name')
    recent = df.loc[grouped.year.idxmax()].reset_index(drop=True)
    recent = recent[['university_name', 'world_rank']]
    return recent

In [ ]:
recent_data = pd.DataFrame(columns=['university_name'])

In [88]:
for rankname, ranking in ranks.items():
    single_recent = get_recent(ranking)
    single_recent.rename(columns={'world_rank': f'rank_{rankname}'}, inplace=True)
    recent_data = recent_data.merge(single_recent, on='university_name', how='outer')

In [89]:
recent_data

,university_name,rank_times,rank_cwur,rank_shanghai
0,AGH University of Science and Technology,601-800,782,NaN
1,Aalborg University,201-250,565,301-400
2,Aalto University,251-300,421,401-500
3,Aarhus University,=106,122,73
4,Aberystwyth University,301-350,814,NaN
...,...,...,...,...
1447,Vrije Universiteit Brussel (VUB),NaN,NaN,201-300
1448,Vrije University Brussel,NaN,NaN,301-400
1449,Washington State University - Pullman,NaN,NaN,201-300
1450,Western University,NaN,NaN,201-300


In [124]:
recent_data_ranks = recent_data[['rank_times', 'rank_cwur', 'rank_shanghai']]

The fuction for computing the (absolute) difference between two ranks is below.

If any of the two ranks is not available, the result is also `NA`.

If any of the two ranks is an interval:
* If there is any overlap between the two ranks it considers the difference as 0. *i.e.* there is no info available to compute a difference.
* Otherwise, it computes the *maximum* difference between the extremes of the two intervals. The function could be trivially changed to compute the *minimum* difference with no consequences.

These assumptions also make the computation symmetric, simplifying the implementation.

In [422]:
def parse_ranks(rank1, rank2):
    if "-" in rank1:
        rank1 = tuple(map(int, rank1.split('-')))
    else:
        rank1 = (int(rank1), int(rank1))
    if "-" in rank2:
        rank2 = tuple(map(int, rank2.split('-')))
    else:
        rank2 = (int(rank2), int(rank2))
    return rank1, rank2


def is_overlapping(rank1, rank2):
    overlap = rank1[0] <= rank2[1] and rank2[0] <= rank1[1]
    if overlap:
        return True
    return False
    

def max_rankdiff(ranks):
    rank1, rank2 = ranks
    if pd.isna(rank1) or pd.isna(rank2):
        return None
    rank1, rank2 = parse_ranks(rank1, rank2)
    if is_overlapping(rank1, rank2):
        return 0
    return abs(rank2[1] - rank1[0])

Now the function is applied to all the possible two ranks combinations.

In [192]:
combs = combinations(recent_data_ranks.columns, 2)l

for comb in combs:
    recent_data['_'.join([colname for colname in comb])] = recent_data_ranks[[colname for colname in comb]] \
                                                           .apply(max_rankdiff, axis=1)

In [193]:
recent_data

,university_name,rank_times,rank_cwur,rank_shanghai,rank_times_rank_cwur,rank_times_rank_shanghai,rank_cwur_rank_shanghai
0,AGH University of Science and Technology,601-800,782,NaN,0.0,NaN,NaN
1,Aalborg University,201-250,565,301-400,364.0,199.0,165.0
2,Aalto University,251-300,421,401-500,170.0,249.0,0.0
3,Aarhus University,106,122,73,16.0,33.0,49.0
4,Aberystwyth University,301-350,814,NaN,513.0,NaN,NaN
...,...,...,...,...,...,...,...
1447,Vrije Universiteit Brussel (VUB),NaN,NaN,201-300,NaN,NaN,NaN
1448,Vrije University Brussel,NaN,NaN,301-400,NaN,NaN,NaN
1449,Washington State University - Pullman,NaN,NaN,201-300,NaN,NaN,NaN
1450,Western University,NaN,NaN,201-300,NaN,NaN,NaN


Printed below is, for each university, the comparison which generated the maximum difference, according to the above criteria, and the resulting difference.

(I am not allowing for ties here since the actual requirement is just to print the maximum *value* among the differences.)

In [291]:
recent_data_rankdiffs_cols = recent_data.columns[-3:]
maxdiff_col = recent_data[recent_data_rankdiffs_cols].idxmax(axis=1)
maxdiff = recent_data[recent_data_rankdiffs_cols].max(axis=1).astype('Int32')

In [292]:
maxdiff_unis = pd.DataFrame({'university': recent_data['university_name'], 'comp': maxdiff_col, 'maxdiff': maxdiff})
maxdiff_unis

,university,comp,maxdiff
0,AGH University of Science and Technology,rank_times_rank_cwur,0
1,Aalborg University,rank_times_rank_cwur,364
2,Aalto University,rank_times_rank_shanghai,249
3,Aarhus University,rank_cwur_rank_shanghai,49
4,Aberystwyth University,rank_times_rank_cwur,513
...,...,...,...
1447,Vrije Universiteit Brussel (VUB),NaN,<NA>
1448,Vrije University Brussel,NaN,<NA>
1449,Washington State University - Pullman,NaN,<NA>
1450,Western University,NaN,<NA>


## Consider only the most recent data point of the *times* dataset. Compute the number of male and female students for each country.

In [341]:
recent_idx_country = ranks['times'].groupby('country')['year'].transform(max) == ranks['times']['year']
recent_country = ranks['times'].loc[recent_idx_country].reset_index(drop=True)

In [342]:
def parse_ratio(ratio):
    if pd.isna(ratio):
        return None
    female_part = ratio.split(' : ')[0]
    return float(female_part) / 100
    

recent_country['female_ratio_parsed'] = recent_country['female_male_ratio'].apply(parse_ratio)

The total number of students is for some reason a string with comma as thousands separator.

In [343]:
recent_country['num_students'] = recent_country['num_students'].str.replace(',', '').astype(float)

In [348]:
def compute_female_male_students(group):
    female_students = round(group.num_students * group.female_ratio_parsed)
    male_students = group.num_students - female_students
    return pd.Series({'num_female': female_students.sum(min_count=1),
                      'num_male': male_students.sum(min_count=1)},
                    dtype='Int64')

recent_grouped_country = recent_country.groupby('country')
sex_counts = recent_grouped_country.apply(compute_female_male_students)
sex_counts

,num_female,num_male
country,,
Argentina,67191,41182
Australia,391736,321640
Austria,68364,66113
Bangladesh,21323,41393
Belarus,20219,9084
...,...,...
Unisted States of America,<NA>,<NA>
United Arab Emirates,9516,4931
United Kingdom,711815,613027


There are some evident typos in some country names. A more thorough work would have corrected them, but this kind of operation (fuzzy match) is computationally expensive so I am skipping it.

## Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [333]:
avgratio = recent_country['female_ratio_parsed'].mean()
avgratio

0.4953804347826086

In [437]:
under_ratio = recent_country[recent_country['female_ratio_parsed'] < avgratio]
under_ratio_relevant_cols = under_ratio[['university_name', 'female_ratio_parsed']]
under_ratio_relevant_cols

,university_name,female_ratio_parsed
0,California Institute of Technology,0.33
1,University of Oxford,0.46
2,Stanford University,0.42
3,University of Cambridge,0.46
4,Massachusetts Institute of Technology,0.37
...,...,...
794,Xidian University,0.29
795,Yeungnam University,0.48
796,Yıldız Technical University,0.36
798,Yokohama National University,0.28


## For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

In [436]:
def get_fraction_in_under_ratio(group):
    den = group['num_students'].sum(min_count=1)
    under_ratio_country = under_ratio[under_ratio['country'] == group.name]
    in_under_ratio = group['university_name'].isin(under_ratio_country['university_name'])
    num = (group['num_students'] * in_under_ratio).sum()
    return num/den

num_in_under_ratio = recent_grouped_country.apply(get_fraction_in_under_ratio)
num_in_under_ratio.to_frame(name='students_in_under_ratio')

,students_in_under_ratio
country,
Argentina,0.000000
Australia,0.146438
Austria,0.398633
Bangladesh,1.000000
Belarus,0.000000
...,...
Unisted States of America,NaN
United Arab Emirates,0.361736
United Kingdom,0.139364


## Read the file `educational_attainment_supplementary_data.csv`, discarding any row with missing `country_name` or `series_name`

In [240]:
raw_attainment = pd.read_csv(os.path.join('data', 'educational_attainment_supplementary_data.csv'),
                             index_col = ['country_name', 'series_name'])

In [246]:
attainment_no_na = raw_attainment.loc[raw_attainment.index.dropna()]

## From `attainment` build a dataframe with the same data, but with 4 columns: `country_name`, `series_name`, `year`, `value`

In [439]:
attainment_long = attainment_no_na.melt(var_name='year', ignore_index=False).reset_index()
attainment_long

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54
...,...,...,...,...
2134345,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134346,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134347,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN
2134348,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN


## For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [441]:
def uni_in_ranking(df):
    unicol = 'university_name' if 'university_name' in df.columns else 'institution'
    return df[unicol].unique()

rankcounts = Counter()

for df in ranks.values():
    in_ranking = uni_in_ranking(df)
    rankcounts.update(in_ranking)

pd.DataFrame.from_dict(rankcounts, columns=['rank_counts'], orient='index')

,rank_counts
Harvard University,3
California Institute of Technology,3
Massachusetts Institute of Technology,2
Stanford University,3
Princeton University,3
...,...
"University at Buffalo, the State University of New York",1
China Medical University,1
Paris Dauphine University (Paris 9),1
"The University of Newcastle, Australia",1


## In the times ranking, compute the number of times each university appears

In [258]:
counts = ranks['times'].groupby('university_name').size().to_frame(name='count')

In [259]:
counts

,count
university_name,
AGH University of Science and Technology,1
Aalborg University,5
Aalto University,5
Aarhus University,6
Aberystwyth University,5
...,...
École Normale Supérieure,6
École Normale Supérieure de Lyon,6
École Polytechnique,6


## Find the universities that appear at most twice in the times ranking.

In [235]:
counts[counts['count'] <= 2]

,count
university_name,
AGH University of Science and Technology,1
Adam Mickiewicz University,1
Aix-Marseille University,1
Ajou University,1
Alexandru Ioan Cuza University,1
...,...
Yokohama City University,1
Yokohama National University,1
Yuan Ze University,2


## The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

I will re-use some function and ideas from point [4](#For-each-ranking,-consider-only-the-most-recent-data-point.-For-each-university,-compute-the-maximum-difference-between-the-rankings-(e.g.-for-Aarhus-University-the-value-is-122-73=49).-Notice-that-some-rankings-are-expressed-as-a-range).

In [445]:
global_df = pd.DataFrame(columns=['university_name', 'year'])

In [446]:
for rankname, ranking in ranks.items():
    ranking_uniform = uniform_df(ranking)
    ranking_needed_cols = ranking_uniform[['university_name', 'year', 'world_rank']].copy()
    ranking_needed_cols.rename(columns={'world_rank': f'rank_{rankname}'}, inplace=True)
    global_df = global_df.merge(ranking_needed_cols, on=['university_name', 'year'], how='right')

global_df = global_df.dropna().reset_index(drop=True)

In [447]:
global_df

,university_name,year,rank_times,rank_cwur,rank_shanghai
0,Harvard University,2012,2,1,1
1,Stanford University,2012,2,3,2
2,"University of California, Berkeley",2012,10,10,4
3,University of Cambridge,2012,6,4,5
4,California Institute of Technology,2012,1,5,6
...,...,...,...,...,...
546,University of Konstanz,2015,226-250,490,401-500
547,University of Pavia,2015,251-275,327,401-500
548,University of Trieste,2015,201-225,342,401-500
549,Vienna University of Technology,2015,226-250,427,401-500


In [448]:
def is_overlapping_wrapper(ranks):
    rank1, rank2 = ranks
    rank1, rank2 = parse_ranks(rank1, rank2)
    return is_overlapping(rank1, rank2)

In [449]:
combs = combinations(global_df.columns[-3:], 2)

samerank_mask = True

for comb in combs:
    samerank_mask &= global_df[[colname for colname in comb]].apply(is_overlapping_wrapper, axis=1)

In [450]:
same_ranks = global_df[samerank_mask]
same_ranks.reset_index(drop=True)

,university_name,year,rank_times,rank_cwur,rank_shanghai
0,Stanford University,2013,2,2,2
1,University of Southampton,2014,146,146,101-150
2,Temple University,2014,351-400,390,301-400
3,University of Vermont,2014,301-350,320,301-400
4,Autonomous University of Barcelona,2015,226-250,241,201-300
5,Laval University,2015,226-250,229,201-300
6,Medical University of Vienna,2015,251-275,265,201-300
7,Newcastle University,2015,201-225,212,201-300
8,University of Waterloo,2015,251-275,269,201-300
9,University of Turku,2015,351-400,383,301-400
